In [1]:
import pysam
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import itertools

In [2]:
bam_file = "/home/dbeb/btech/bb1160039/scratch/project/6987_possorted_genome_bam.bam"
bai_file = "/home/dbeb/btech/bb1160039/scratch/project/6987_possorted_genome_bam.bam.bai"

In [3]:
samf = pysam.Samfile(bam_file, "rb")
replicon_dict = dict([[replicon, {'seq_start_pos': 0,'seq_end_pos': length}] for replicon, length in zip(samf.references, samf.lengths)])

In [4]:
print(replicon_dict['1']['seq_start_pos'])
print(replicon_dict['1']['seq_end_pos'])

0
248956422


In [5]:
samfile = pysam.AlignmentFile(bam_file, "rb", index_filename = bai_file)

In [6]:
# x=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','X','Y']
x=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','X','Y']
list_tags = []

for i in tqdm(range(0,len(x))):
    for read in samfile.fetch(x[i], replicon_dict[x[i]]['seq_start_pos'],replicon_dict[x[i]]['seq_end_pos']):
        try:
            if read.has_tag("GX") and read.get_tag("NH")==1:
                list_tags.append([read.get_tag("CB"),read.get_tag("UB"),str(read.get_tag("GX")+"-"+read.reference_name+"-"+str(read.get_reference_positions()[0])+"-"+str(len(read.get_reference_positions())))])
        except KeyError:
            continue

In [7]:
%%time
list_tags_rm_dup = list(list_tags for list_tags,_ in itertools.groupby(list_tags))

CPU times: user 5.08 s, sys: 1.08 s, total: 6.16 s
Wall time: 6.17 s


In [8]:
print(len(list_tags))
print(len(list_tags_rm_dup))

25258957
20656024


In [9]:
list_tags_rm_dup

[['CAGTGCGGTTCTCCTG-1', 'TCCTTGTTACCT', 'ENSG00000238009-1-129117-71'],
 ['CAGTGCGGTTCTCCTG-1', 'TCCTTGTTACCT', 'ENSG00000238009-1-129118-70'],
 ['GGGTCACCAGCCGGTT-1', 'ATGTCGTTCCTT', 'ENSG00000237491-1-778888-71'],
 ['GGGTCACCAGCCGGTT-1', 'ATGTCGTTCCTT', 'ENSG00000237491-1-778888-76'],
 ['TATCCTAGTTCCATTT-1', 'AAATTCTGCGGC', 'ENSG00000237491-1-781973-78'],
 ['GCGGATCGTGGATACG-1', 'CCCATACTTACA', 'ENSG00000237491-1-781976-75'],
 ['CACTGGGTCGTCTAAG-1', 'TGTAACGGCATG', 'ENSG00000237491-1-781993-71'],
 ['CACTGGGTCGTCTAAG-1', 'TGTAACGGCATG', 'ENSG00000237491-1-781993-73'],
 ['CACTGGGTCGTCTAAG-1', 'TGTAACGGCATG', 'ENSG00000237491-1-781993-71'],
 ['CCCTCTCGTTTGCAGT-1', 'ACGCCCCCCACT', 'ENSG00000237491-1-781993-76'],
 ['CCCTCTCGTTTGCAGT-1', 'ACGCCCCCCACT', 'ENSG00000237491-1-781993-71'],
 ['TGAGGAGAGAGAACCC-1', 'GAATGCTATCAT', 'ENSG00000237491-1-781993-74'],
 ['ACCTGTCCACAGCCTG-1', 'TCTACGGTTAGA', 'ENSG00000237491-1-782040-101'],
 ['AAAGGTAAGCTGTGCC-1', 'CAAGTTTATATA', 'ENSG00000237491-1-7820

In [10]:
start=0
val=int(list_tags_rm_dup[start][2].split("-")[2])

for i in tqdm(range(1,len(list_tags_rm_dup))):
    if (int(list_tags_rm_dup[i][2].split("-")[2]) - val>50):
        if (i-start>1):
            #update the inner elements
            for inner in list_tags_rm_dup[start:i]:
                inner[2] = list_tags_rm_dup[start][2]
        #update start to point to this pos
        start = i
        #update val to the val at this pos
        val = int(list_tags_rm_dup[i][2].split("-")[2])

In [11]:
%%time
list_tags_rm_dup_final = list(list_tags_rm_dup for list_tags_rm_dup,_ in itertools.groupby(list_tags_rm_dup))

CPU times: user 4.2 s, sys: 354 ms, total: 4.55 s
Wall time: 4.55 s


In [12]:
n=len(list_tags_rm_dup_final)
n

20533651

In [13]:
%%time
df_all_p1 = pd.DataFrame(list_tags_rm_dup_final[:int(n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p2 = pd.DataFrame(list_tags_rm_dup_final[int(n/10):int(2*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p3 = pd.DataFrame(list_tags_rm_dup_final[int(2*n/10):int(3*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p4 = pd.DataFrame(list_tags_rm_dup_final[int(3*n/10):int(4*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p5 = pd.DataFrame(list_tags_rm_dup_final[int(4*n/10):int(5*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p6 = pd.DataFrame(list_tags_rm_dup_final[int(5*n/10):int(6*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p7 = pd.DataFrame(list_tags_rm_dup_final[int(6*n/10):int(7*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p8 = pd.DataFrame(list_tags_rm_dup_final[int(7*n/10):int(8*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p9 = pd.DataFrame(list_tags_rm_dup_final[int(8*n/10):int(9*n/10)], columns=["celltag","moltag","pseudoname"])
df_all_p10 = pd.DataFrame(list_tags_rm_dup_final[int(9*n/10):], columns=["celltag","moltag","pseudoname"])

CPU times: user 7.89 s, sys: 458 ms, total: 8.35 s
Wall time: 8.34 s


In [14]:
%%time
c1 = df_all_p1['celltag'].value_counts()
c2 = df_all_p2['celltag'].value_counts()
c3 = df_all_p3['celltag'].value_counts()
c4 = df_all_p4['celltag'].value_counts()
c5 = df_all_p5['celltag'].value_counts()
c6 = df_all_p6['celltag'].value_counts()
c7 = df_all_p7['celltag'].value_counts()
c8 = df_all_p8['celltag'].value_counts()
c9 = df_all_p9['celltag'].value_counts()
c10 = df_all_p10['celltag'].value_counts()
c11 = df_all_p1['pseudoname'].value_counts()
c22 = df_all_p2['pseudoname'].value_counts()
c33 = df_all_p3['pseudoname'].value_counts()
c44 = df_all_p4['pseudoname'].value_counts()
c55 = df_all_p5['pseudoname'].value_counts()
c66 = df_all_p6['pseudoname'].value_counts()
c77 = df_all_p7['pseudoname'].value_counts()
c88 = df_all_p8['pseudoname'].value_counts()
c99 = df_all_p9['pseudoname'].value_counts()
c1010 = df_all_p10['pseudoname'].value_counts()

CPU times: user 16.3 s, sys: 239 ms, total: 16.5 s
Wall time: 16.7 s


In [15]:
%time
df_all_p1_subset = df_all_p1[df_all_p1["celltag"].isin(c1[c1>10].index)]
df_all_p1_subset = df_all_p1_subset[df_all_p1_subset["pseudoname"].isin(c11[c11>10].index)]
df_all_p2_subset = df_all_p2[df_all_p2["celltag"].isin(c2[c2>10].index)]
df_all_p2_subset = df_all_p2_subset[df_all_p2_subset["pseudoname"].isin(c22[c22>10].index)]
df_all_p3_subset = df_all_p3[df_all_p3["celltag"].isin(c3[c3>10].index)]
df_all_p3_subset = df_all_p3_subset[df_all_p3_subset["pseudoname"].isin(c33[c33>10].index)]
df_all_p4_subset = df_all_p4[df_all_p4["celltag"].isin(c4[c4>10].index)]
df_all_p4_subset = df_all_p4_subset[df_all_p4_subset["pseudoname"].isin(c44[c44>10].index)]
df_all_p5_subset = df_all_p5[df_all_p5["celltag"].isin(c5[c5>10].index)]
df_all_p5_subset = df_all_p5_subset[df_all_p5_subset["pseudoname"].isin(c55[c55>10].index)]
df_all_p6_subset = df_all_p6[df_all_p6["celltag"].isin(c6[c6>10].index)]
df_all_p6_subset = df_all_p6_subset[df_all_p6_subset["pseudoname"].isin(c66[c66>10].index)]
df_all_p7_subset = df_all_p7[df_all_p7["celltag"].isin(c7[c7>10].index)]
df_all_p7_subset = df_all_p7_subset[df_all_p7_subset["pseudoname"].isin(c77[c77>10].index)]
df_all_p8_subset = df_all_p8[df_all_p8["celltag"].isin(c8[c8>10].index)]
df_all_p8_subset = df_all_p8_subset[df_all_p8_subset["pseudoname"].isin(c88[c88>10].index)]
df_all_p9_subset = df_all_p9[df_all_p9["celltag"].isin(c9[c9>10].index)]
df_all_p9_subset = df_all_p9_subset[df_all_p9_subset["pseudoname"].isin(c99[c99>10].index)]
df_all_p10_subset = df_all_p10[df_all_p10["celltag"].isin(c10[c10>10].index)]
df_all_p10_subset = df_all_p10_subset[df_all_p10_subset["pseudoname"].isin(c1010[c1010>10].index)]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


In [16]:
%%time
counts_p1=df_all_p1_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p2=df_all_p2_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p3=df_all_p3_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p4=df_all_p4_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p5=df_all_p5_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p6=df_all_p6_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p7=df_all_p7_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p8=df_all_p8_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p9=df_all_p9_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)
counts_p10=df_all_p10_subset.groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0)

CPU times: user 29.7 s, sys: 18.5 s, total: 48.2 s
Wall time: 48.3 s


In [17]:
cell_bcode=set(counts_p1.columns).intersection(set(counts_p2.columns)).intersection(set(counts_p3.columns)).intersection(set(counts_p4.columns)).intersection(set(counts_p5.columns)).intersection(set(counts_p6.columns)).intersection(set(counts_p7.columns)).intersection(set(counts_p8.columns)).intersection(set(counts_p9.columns)).intersection(set(counts_p10.columns))

In [18]:
len(cell_bcode)

20665

In [19]:
%%time
counts_p1 = counts_p1[cell_bcode]
counts_p2 = counts_p2[cell_bcode]
counts_p3 = counts_p3[cell_bcode]
counts_p4 = counts_p4[cell_bcode]
counts_p5 = counts_p5[cell_bcode]
counts_p6 = counts_p6[cell_bcode]
counts_p7 = counts_p7[cell_bcode]
counts_p8 = counts_p8[cell_bcode]
counts_p9 = counts_p9[cell_bcode]
counts_p10 = counts_p10[cell_bcode]

CPU times: user 27.6 s, sys: 6.97 s, total: 34.5 s
Wall time: 34.5 s


In [20]:
%%time
counts_full = pd.concat([counts_p1,counts_p2, counts_p3, counts_p4, counts_p5, counts_p6, counts_p7, counts_p8, counts_p9, counts_p10])

CPU times: user 6.31 s, sys: 6.6 s, total: 12.9 s
Wall time: 12.9 s


In [21]:
# counts_full.to_csv("/home/dbeb/btech/bb1160039/scratch/project/rmmulti/counts_genes_brain_rmmulti.csv")